# Timer

In [1]:
from time import sleep
from datetime import datetime

month, day, hour, minute = 10, 25, 20, 15

while True:
    dt = datetime.now()
    if dt.month >= month and dt.day >= day:
        if dt.hour > hour or (dt.hour == hour and dt.minute > minute):
            break
    sleep(60)

# Parameter optimization

In [2]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'buy'
pattern = ['PriceChange']
work_timeframe = '15m'
higher_timeframe = '4h'
opt_limit = 50
load = False

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {'PriceChange': {"low_price_quantile": [1 * (i + 1) for i in range(20)]}}

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['indicator_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat = stat.sort_values(['indicator_rank', 'pct_right_forecast_avg'], ascending=False)
stat.head(20)

Number of combinations is 20


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [02:51<00:00,  8.58s/it]


,pattern,PriceChange_low_price_quantile,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,pct_right_forecast_4,pct_right_forecast_5,pct_right_forecast_6,pct_right_forecast_7,pct_right_forecast_8,pct_right_forecast_9,pct_right_forecast_10,pct_right_forecast_11,pct_right_forecast_12,pct_right_forecast_13,pct_right_forecast_14,pct_right_forecast_15,pct_right_forecast_16,pct_right_forecast_17,pct_right_forecast_18,pct_right_forecast_19,pct_right_forecast_20,pct_right_forecast_21,pct_right_forecast_22,pct_right_forecast_23,pct_right_forecast_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg,indicator_rank
0,PriceChange,1,100.00,50.00,50.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,50.00,1.40,2.40,0.94,-4.54,-5.25,-5.37,-6.67,-6.82,-6.87,-6.94,-5.10,-5.10,-6.32,-7.03,-7.01,-6.29,-7.36,-7.59,-5.97,-5.45,-5.47,-6.35,-6.60,-3.76,2,10.416667,-119.166667
1,PriceChange,2,75.00,75.00,75.00,75.00,25.00,25.00,25.00,0.00,0.00,0.00,25.00,25.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.24,0.67,0.22,0.75,-3.57,-3.84,-3.96,-5.68,-5.97,-5.69,-5.58,-4.37,-4.62,-5.25,-5.83,-5.31,-5.89,-6.69,-7.31,-5.54,-5.61,-6.86,-6.15,-6.24,4,17.708333,-209.166667
2,PriceChange,3,80.00,80.00,80.00,80.00,40.00,40.00,40.00,20.00,20.00,20.00,40.00,20.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.66,0.89,0.14,1.48,-1.35,-1.07,-1.07,-2.50,-2.80,-2.14,-1.48,-6.22,-6.45,-7.30,-8.60,-9.14,-9.23,-9.68,-9.68,-6.45,-5.85,-6.99,-7.53,-6.99,5,23.333333,-233.333333
3,PriceChange,4,80.00,80.00,80.00,80.00,40.00,40.00,40.00,20.00,20.00,20.00,40.00,20.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.66,0.89,0.14,1.48,-1.35,-1.07,-1.07,-2.50,-2.80,-2.14,-1.48,-6.22,-6.45,-7.30,-8.60,-9.14,-9.23,-9.68,-9.68,-6.45,-5.85,-6.99,-7.53,-6.99,5,23.333333,-233.333333
4,PriceChange,5,85.71,71.43,57.14,71.43,71.43,42.86,42.86,42.86,28.57,42.86,28.57,28.57,14.29,14.29,14.29,14.29,14.29,14.29,14.29,14.29,0.00,0.00,0.00,0.00,0.91,0.65,0.09,1.04,0.42,-0.53,-1.40,-0.89,-0.89,-1.18,-1.48,-1.91,-4.19,-4.68,-4.87,-4.96,-3.19,-4.45,-5.59,-6.45,-5.38,-6.99,-7.53,-6.64,7,30.358750,-277.488750
12,PriceChange,13,92.86,71.43,64.29,71.43,71.43,57.14,57.14,50.00,57.14,57.14,35.71,42.86,28.57,21.43,28.57,28.57,28.57,21.43,21.43,28.57,28.57,35.71,28.57,28.57,1.27,0.77,0.20,0.95,0.98,0.64,0.63,0.38,0.31,0.15,-1.21,-1.26,-1.78,-1.92,-0.98,-1.49,-1.74,-1.56,-1.52,-1.84,-1.01,-0.55,-1.02,-0.91,14,44.047083,-363.340833
5,PriceChange,6,87.50,50.00,37.50,50.00,50.00,37.50,37.50,25.00,25.00,25.00,25.00,25.00,12.50,12.50,12.50,12.50,12.50,12.50,12.50,12.50,0.00,0.00,0.00,0.00,0.98,0.18,-1.68,-0.18,-0.74,-1.24,-2.81,-2.72,-2.74,-2.65,-3.67,-3.38,-4.26,-4.45,-4.87,-5.75,-5.96,-5.50,-5.55,-5.69,-4.78,-5.66,-6.21,-5.72,8,23.958333,-368.333333
6,PriceChange,7,87.50,50.00,37.50,50.00,50.00,37.50,37.50,25.00,25.00,25.00,25.00,25.00,12.50,12.50,12.50,12.50,12.50,12.50,12.50,12.50,0.00,0.00,0.00,0.00,0.98,0.18,-1.68,-0.18,-0.74,-1.24,-2.81,-2.72,-2.74,-2.65,-3.67,-3.38,-4.26,-4.45,-4.87,-5.75,-5.96,-5.50,-5.55,-5.69,-4.78,-5.66,-6.21,-5.72,8,23.958333,-368.333333
7,PriceChange,8,88.89,55.56,44.44,55.56,55.56,44.44,44.44,33.33,33.33,33.33,33.33,33.33,11.11,11.11,22.22,11.11,11.11,11.11,11.11,11.11,0.00,11.11,11.11,11.11,1.05,0.37,-1.51,1.04,0.42,0.00,-2.65,-2.43,-1.08,-2.59,-3.54,-2.82,-4.19,-4.21,-4.87,-5.08,-5.28,-4.88,-5.51,-4.93,-4.19,-4.34,-4.90,-4.80,9,28.702500,-371.677500
13,PriceChange,14,93.33,73.33,66.67,73.33,66.67,53.33,53.33,46.67,60.00,60.00,40.00,46.67,26.67,20.00,26.67,26.67,26.67,2

# Save new config data to config file

In [4]:
from config_updater import ConfigUpdater

timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'PriceChange': {"low_price_quantile": [10]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)